In [4]:
import numpy as np
import cv2 as cv

In [5]:
def isolateWindows(filename, sizeCutoff=600, xmin=100, xmax=1200):
  """
  Inputs - filename, a file location, sizecutoff, the minimum area of a contour to be considered and
  xmin and xmax, the bounds of where the windows are tested for
  Outputs - a dictionary of all five windows, formatted "xmin", "ymin", "leng", and "hei".
  Purpose - Find the five windows at the bottom of a 5CSRTT cage for mice. Often needs to be sanity
  checked 
  """
  cens = {0: {
              "xmin": None,
              "ymin": None,
              "leng": None,
              "hei": None,
            },
          1: {
              "xmin": None,
              "ymin": None,
              "leng": None,
              "hei": None,
            },
          2: {
              "xmin": None,
              "ymin": None,
              "leng": None,
              "hei": None,
            },
          3: {
              "xmin": None,
              "ymin": None,
              "leng": None,
              "hei": None,
            },
          4: {
              "xmin": None,
              "ymin": None,
              "leng": None,
              "hei": None,
            },
          }



  cap = cv.VideoCapture(filename)

  contourHistory = -1
  while(True):

    ret, frame = cap.read()

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray,(5,5),0)
    ret, thresh_img = cv.threshold(blur,70,255,cv.THRESH_BINARY)

    contours =  cv.findContours(thresh_img,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)[-2]

    contours = contours[:20]

    contourList = []
    for i, c in enumerate(contours):
      if cv.contourArea(c) > sizeCutoff and c[:, :, 0].max() < xmax and c[:, :, 0].min() > xmin:
        contourList.append(np.array([cv.contourArea(c), i, (c[:, :, 0].max()+c[:, :, 0].min())/2-frame.shape[1]/2]))
    contourList = np.array(contourList)

    sorte = np.argsort(contourList[:, 2])
    sorte = contourList[sorte[:5]]

    for index in sorte[:, 1]:
      cv.drawContours(frame, contours[:30][int(index)], -1, (0,255,0), 3)
    if contourHistory == -1:
      prevContour = sorte[:, 1]
      contourHistory = 0
    elif contourHistory < 5 and np.array_equal(sorte[:, 1], prevContour):
      contourHistory += 1
    elif contourHistory >= 5:
      print("Done!")
      break
    else:
      contourHistory = -1
      
    cv.imshow('video feed',frame)
    if cv.waitKey(1) == ord('q'):
      break
  # indexes = np.sort(sorte[:, 1])
  
  for count, i in enumerate(sorte[:, 1]):
    # print(contours[:30][int(i)])
    cens[count]["xmin"] = contours[:30][int(i)][:, 0, 0].min()
    cens[count]["ymin"] = contours[:30][int(i)][:, 0, 1].min()
    cens[count]["leng"] = contours[:30][int(i)][:, 0, 0].max() - contours[:30][int(i)][:, 0, 0].min()
    cens[count]["hei"] = contours[:30][int(i)][:, 0, 1].max() - contours[:30][int(i)][:, 0, 1].min()
  cap.release()
  cv.destroyAllWindows()
  return cens

In [6]:
filename = r'C:\Users\naimc\Desktop\work\final\data\2C_Mouse_5-Choice_MustTouchTraining_a1_greyscale.mp4'
isolateWindows(filename, xmin=300, xmax=1100)

Done!


{0: {'xmin': 392, 'ymin': 661, 'leng': 95, 'hei': 16},
 1: {'xmin': 521, 'ymin': 669, 'leng': 124, 'hei': 16},
 2: {'xmin': 678, 'ymin': 675, 'leng': 127, 'hei': 14},
 3: {'xmin': 838, 'ymin': 677, 'leng': 119, 'hei': 12},
 4: {'xmin': 994, 'ymin': 676, 'leng': 81, 'hei': 10}}